# Quantum ESPRESSO Workshop - Part 5: Lattice Parameter Optimization

## Learning Objectives
1. Understand why lattice optimization is necessary
2. Perform energy vs volume calculations (equation of state)
3. Fit the Birch-Murnaghan equation of state
4. Extract equilibrium lattice parameter and bulk modulus
5. Compare with experimental values

---

## 1. Why Optimize the Lattice Parameter?

### The Problem

DFT with different exchange-correlation functionals gives different equilibrium volumes:
- **LDA**: Typically underestimates lattice parameters (~1-2%)
- **PBE (GGA)**: Typically overestimates lattice parameters (~1-2%)

### Why It Matters

Using the experimental lattice parameter with DFT introduces **internal stress**:
- Affects calculated properties (band gaps, phonons, etc.)
- Not self-consistent with the chosen functional

### The Solution

**Always optimize the lattice parameter** with converged ecutwfc and k-points before calculating any properties.

### Methods

1. **Variable-cell relaxation** (`vc-relax`): Let QE optimize automatically
2. **Energy vs volume scan**: Calculate E(V) and fit equation of state

We'll use method 2 as it's more instructive and gives additional properties (bulk modulus).

---

## 2. Setup

In [ ]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.optimize import curve_fit
import re
import time
import json

# Paths
WORKSHOP_ROOT = Path('/home/claude/qe_workshop')
PSEUDO_DIR = WORKSHOP_ROOT / 'pseudopotentials'
OUTPUT_DIR = WORKSHOP_ROOT / 'outputs'

# Working directory
WORK_DIR = OUTPUT_DIR / '05_lattice_optimization'
WORK_DIR.mkdir(exist_ok=True)
(WORK_DIR / 'tmp').mkdir(exist_ok=True)

print(f"Working directory: {WORK_DIR}")

In [ ]:
# Load converged parameters
params_file = WORKSHOP_ROOT / 'converged_parameters.json'

if params_file.exists():
    with open(params_file, 'r') as f:
        params = json.load(f)
    ecutwfc = params.get('ecutwfc_recommended', 40.0)
    ecutrho_factor = params.get('ecutrho_factor', 8)
    kgrid = params.get('kpoints_recommended', 8)
    print("Loaded converged parameters:")
    print(f"  ecutwfc = {ecutwfc} Ry")
    print(f"  ecutrho = {ecutrho_factor} × ecutwfc")
    print(f"  k-points = {kgrid}×{kgrid}×{kgrid}")
else:
    ecutwfc = 40.0
    ecutrho_factor = 8
    kgrid = 8
    print("Using default parameters")

ecutrho = ecutwfc * ecutrho_factor

---

## 3. Helper Functions

In [ ]:
def generate_scf_input(prefix, ecutwfc, ecutrho, kpoints, pseudo_dir,
                       celldm1, conv_thr=1.0e-8):
    """Generate SCF input with variable lattice parameter."""
    kx, ky, kz = kpoints
    
    input_text = f"""&CONTROL
    calculation = 'scf'
    prefix = '{prefix}'
    outdir = './tmp'
    pseudo_dir = '{pseudo_dir}'
    verbosity = 'high'
    tprnfor = .true.
    tstress = .true.
/

&SYSTEM
    ibrav = 2
    celldm(1) = {celldm1}
    nat = 2
    ntyp = 1
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'smearing'
    smearing = 'cold'
    degauss = 0.01
/

&ELECTRONS
    conv_thr = {conv_thr}
    mixing_beta = 0.7
/

ATOMIC_SPECIES
    Si  28.0855  Si.upf

ATOMIC_POSITIONS {{crystal}}
    Si  0.00  0.00  0.00
    Si  0.25  0.25  0.25

K_POINTS {{automatic}}
    {kx} {ky} {kz} 0 0 0
"""
    return input_text


def parse_output(output_text):
    """Parse energy, volume, and pressure from output."""
    results = {}
    
    for line in output_text.split('\n'):
        if '!' in line and 'total energy' in line:
            match = re.search(r'=\s+([\d.E+-]+)\s+Ry', line)
            if match:
                results['energy_ry'] = float(match.group(1))
        
        if 'unit-cell volume' in line:
            match = re.search(r'=\s+([\d.]+)', line)
            if match:
                results['volume_bohr3'] = float(match.group(1))
        
        if 'total   stress' in line and 'P=' in line:
            match = re.search(r'P=\s*([\d.E+-]+)', line)
            if match:
                results['pressure_kbar'] = float(match.group(1))
    
    results['converged'] = 'convergence has been achieved' in output_text
    
    return results


def run_pwscf(input_file, timeout=300):
    """Run pw.x calculation."""
    input_file = Path(input_file)
    output_file = input_file.with_suffix('.out')
    work_dir = input_file.parent
    
    start = time.time()
    result = subprocess.run(
        ['pw.x', '-in', input_file.name],
        capture_output=True,
        text=True,
        cwd=work_dir,
        timeout=timeout
    )
    elapsed = time.time() - start
    
    with open(output_file, 'w') as f:
        f.write(result.stdout)
    
    return result.stdout, elapsed

---

## 4. Energy vs Volume Calculation

We scan lattice parameters around the experimental value (10.26 Bohr = 5.43 Å).

In [ ]:
# Define lattice parameter range
# Experimental: a = 5.43 Å = 10.26 Bohr
a_exp_bohr = 10.26

# Scan ±4% around experimental value
celldm1_values = np.linspace(a_exp_bohr * 0.96, a_exp_bohr * 1.04, 9)

print("Lattice Parameter Scan")
print("=" * 50)
print(f"Experimental lattice parameter: {a_exp_bohr} Bohr = {a_exp_bohr * 0.529177:.4f} Å")
print(f"\nScan range: {celldm1_values[0]:.4f} - {celldm1_values[-1]:.4f} Bohr")
print(f"            {celldm1_values[0] * 0.529177:.4f} - {celldm1_values[-1] * 0.529177:.4f} Å")
print(f"Number of points: {len(celldm1_values)}")

In [ ]:
# Run E(V) calculations
results = []

print("\nRunning E(V) calculations...")
print("=" * 90)
print(f"{'celldm(1) (Bohr)':<18} {'a (Å)':<12} {'V (Å³)':<12} {'E (Ry)':<18} {'P (kbar)':<12} {'Status'}")
print("-" * 90)

kpoints = (kgrid, kgrid, kgrid)

for celldm1 in celldm1_values:
    prefix = f'si_a{celldm1:.3f}'.replace('.', 'p')
    
    # Generate and write input
    input_text = generate_scf_input(
        prefix=prefix,
        ecutwfc=ecutwfc,
        ecutrho=ecutrho,
        kpoints=kpoints,
        pseudo_dir=PSEUDO_DIR,
        celldm1=celldm1
    )
    
    input_file = WORK_DIR / f'{prefix}.in'
    with open(input_file, 'w') as f:
        f.write(input_text)
    
    # Run calculation
    output, elapsed = run_pwscf(input_file)
    parsed = parse_output(output)
    
    # Convert units
    a_angstrom = celldm1 * 0.529177
    v_angstrom3 = parsed.get('volume_bohr3', 0) * 0.529177**3 if parsed.get('volume_bohr3') else None
    
    status = '✓' if parsed.get('converged', False) else '✗'
    e_str = f"{parsed.get('energy_ry', 0):.8f}" if parsed.get('energy_ry') else 'N/A'
    p_str = f"{parsed.get('pressure_kbar', 0):.2f}" if parsed.get('pressure_kbar') is not None else 'N/A'
    v_str = f"{v_angstrom3:.4f}" if v_angstrom3 else 'N/A'
    
    print(f"{celldm1:<18.4f} {a_angstrom:<12.4f} {v_str:<12} {e_str:<18} {p_str:<12} {status}")
    
    results.append({
        'celldm1_bohr': celldm1,
        'a_angstrom': a_angstrom,
        'volume_bohr3': parsed.get('volume_bohr3'),
        'volume_angstrom3': v_angstrom3,
        'energy_ry': parsed.get('energy_ry'),
        'pressure_kbar': parsed.get('pressure_kbar'),
        'converged': parsed.get('converged', False)
    })

print("=" * 90)
print("E(V) calculations complete!")

In [ ]:
# Save results
results_file = WORK_DIR / 'ev_curve_results.json'
with open(results_file, 'w') as f:
    json.dump(results, f, indent=2)
print(f"Results saved to: {results_file}")

---

## 5. Birch-Murnaghan Equation of State

The third-order Birch-Murnaghan equation of state:

$$E(V) = E_0 + \frac{9V_0B_0}{16}\left\{\left[\left(\frac{V_0}{V}\right)^{2/3}-1\right]^3 B_0' + \left[\left(\frac{V_0}{V}\right)^{2/3}-1\right]^2 \left[6-4\left(\frac{V_0}{V}\right)^{2/3}\right]\right\}$$

where:
- $E_0$: Equilibrium energy
- $V_0$: Equilibrium volume
- $B_0$: Bulk modulus
- $B_0'$: Pressure derivative of bulk modulus (typically ~4)

In [ ]:
def birch_murnaghan(V, E0, V0, B0, B0_prime):
    """
    Third-order Birch-Murnaghan equation of state.
    
    Parameters
    ----------
    V : array
        Volume
    E0 : float
        Equilibrium energy
    V0 : float
        Equilibrium volume
    B0 : float
        Bulk modulus
    B0_prime : float
        Pressure derivative of bulk modulus
    
    Returns
    -------
    E : array
        Energy at given volumes
    """
    eta = (V0 / V) ** (2.0 / 3.0)
    E = E0 + (9.0 * V0 * B0 / 16.0) * (
        (eta - 1.0) ** 3 * B0_prime + 
        (eta - 1.0) ** 2 * (6.0 - 4.0 * eta)
    )
    return E


def birch_murnaghan_pressure(V, V0, B0, B0_prime):
    """
    Pressure from Birch-Murnaghan EOS.
    P = -dE/dV
    """
    eta = (V0 / V) ** (2.0 / 3.0)
    P = (3.0 * B0 / 2.0) * (eta ** (7.0 / 2.0) - eta ** (5.0 / 2.0)) * (
        1.0 + 0.75 * (B0_prime - 4.0) * (eta - 1.0)
    )
    return P

In [ ]:
# Extract converged data
converged_data = [r for r in results if r['converged'] and r['energy_ry'] is not None]

if len(converged_data) < 5:
    print("ERROR: Not enough data points for fitting!")
else:
    V_data = np.array([r['volume_bohr3'] for r in converged_data])
    E_data = np.array([r['energy_ry'] for r in converged_data])
    a_data = np.array([r['celldm1_bohr'] for r in converged_data])
    
    # Initial guesses
    E0_guess = E_data.min()
    V0_guess = V_data[E_data.argmin()]
    B0_guess = 100.0  # GPa, convert to Ry/Bohr³ later
    B0_prime_guess = 4.0
    
    # Convert B0 guess to Ry/Bohr³
    # 1 GPa = 1e9 Pa = 1e9 J/m³
    # 1 Ry = 2.1798741e-18 J
    # 1 Bohr = 5.29177e-11 m
    # 1 GPa = 1e9 * (5.29177e-11)³ / 2.1798741e-18 Ry/Bohr³
    gpa_to_ry_bohr3 = 1.0 / 14710.507  # conversion factor
    B0_guess_ry = B0_guess * gpa_to_ry_bohr3
    
    print(f"Initial guesses:")
    print(f"  E0 = {E0_guess:.6f} Ry")
    print(f"  V0 = {V0_guess:.4f} Bohr³")
    print(f"  B0 = {B0_guess} GPa = {B0_guess_ry:.6f} Ry/Bohr³")
    print(f"  B0' = {B0_prime_guess}")

In [ ]:
# Fit the Birch-Murnaghan EOS
p0 = [E0_guess, V0_guess, B0_guess_ry, B0_prime_guess]

popt, pcov = curve_fit(birch_murnaghan, V_data, E_data, p0=p0)

E0_fit, V0_fit, B0_fit_ry, B0_prime_fit = popt
perr = np.sqrt(np.diag(pcov))

# Convert B0 to GPa
B0_fit_gpa = B0_fit_ry / gpa_to_ry_bohr3

# Calculate equilibrium lattice parameter
# For FCC: V = a³/4 (primitive cell with 2 atoms)
# So a = (4V)^(1/3)
a0_fit_bohr = (4 * V0_fit) ** (1.0 / 3.0)
a0_fit_angstrom = a0_fit_bohr * 0.529177

print("\nBirch-Murnaghan Fit Results")
print("=" * 60)
print(f"Equilibrium energy:     E0 = {E0_fit:.8f} Ry")
print(f"Equilibrium volume:     V0 = {V0_fit:.4f} Bohr³")
print(f"                           = {V0_fit * 0.529177**3:.4f} ų")
print(f"Bulk modulus:          B0 = {B0_fit_gpa:.2f} GPa")
print(f"Bulk modulus derivative: B0' = {B0_prime_fit:.2f}")
print(f"\nEquilibrium lattice parameter:")
print(f"  a0 = {a0_fit_bohr:.4f} Bohr")
print(f"     = {a0_fit_angstrom:.4f} Å")
print("=" * 60)

In [ ]:
# Compare with experiment
a_exp_angstrom = 5.431  # Experimental lattice parameter
B0_exp_gpa = 97.6       # Experimental bulk modulus

print("\nComparison with Experiment")
print("=" * 60)
print(f"{'Property':<30} {'DFT (PBE)':<15} {'Experiment':<15} {'Error'}")
print("-" * 60)

a_error = (a0_fit_angstrom - a_exp_angstrom) / a_exp_angstrom * 100
B_error = (B0_fit_gpa - B0_exp_gpa) / B0_exp_gpa * 100

print(f"{'Lattice parameter (Å)':<30} {a0_fit_angstrom:<15.4f} {a_exp_angstrom:<15.3f} {a_error:+.2f}%")
print(f"{'Bulk modulus (GPa)':<30} {B0_fit_gpa:<15.2f} {B0_exp_gpa:<15.1f} {B_error:+.2f}%")
print("=" * 60)
print("\nNote: PBE typically overestimates lattice parameters by 1-2%")

---

## 6. Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Generate smooth curve for fit
V_smooth = np.linspace(V_data.min() * 0.98, V_data.max() * 1.02, 200)
E_fit = birch_murnaghan(V_smooth, *popt)

# Plot 1: E(V) curve
ax1 = axes[0, 0]
ax1.plot(V_data, E_data, 'bo', markersize=10, label='DFT data')
ax1.plot(V_smooth, E_fit, 'r-', linewidth=2, label='Birch-Murnaghan fit')
ax1.axvline(x=V0_fit, color='g', linestyle='--', alpha=0.7, label=f'V₀ = {V0_fit:.2f} Bohr³')
ax1.set_xlabel('Volume (Bohr³)', fontsize=12)
ax1.set_ylabel('Total Energy (Ry)', fontsize=12)
ax1.set_title('Energy vs Volume', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: E(a) curve
ax2 = axes[0, 1]
a_smooth = (4 * V_smooth) ** (1.0 / 3.0) * 0.529177  # Convert to Å
a_data_angstrom = a_data * 0.529177
ax2.plot(a_data_angstrom, E_data, 'bo', markersize=10, label='DFT data')
ax2.plot(a_smooth, E_fit, 'r-', linewidth=2, label='Birch-Murnaghan fit')
ax2.axvline(x=a0_fit_angstrom, color='g', linestyle='--', alpha=0.7, 
            label=f'a₀ = {a0_fit_angstrom:.4f} Å')
ax2.axvline(x=a_exp_angstrom, color='orange', linestyle=':', alpha=0.7,
            label=f'Exp. = {a_exp_angstrom} Å')
ax2.set_xlabel('Lattice Parameter (Å)', fontsize=12)
ax2.set_ylabel('Total Energy (Ry)', fontsize=12)
ax2.set_title('Energy vs Lattice Parameter', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Pressure vs Volume
ax3 = axes[1, 0]
P_data = np.array([r['pressure_kbar'] for r in converged_data if r['pressure_kbar'] is not None])
V_p_data = np.array([r['volume_bohr3'] for r in converged_data if r['pressure_kbar'] is not None])

# Calculate fitted pressure (in kbar, 1 GPa = 10 kbar)
P_fit_gpa = birch_murnaghan_pressure(V_smooth, V0_fit, B0_fit_gpa, B0_prime_fit)
P_fit_kbar = P_fit_gpa * 10

if len(P_data) > 0:
    ax3.plot(V_p_data, P_data, 'bo', markersize=10, label='DFT data')
ax3.plot(V_smooth, P_fit_kbar, 'r-', linewidth=2, label='From EOS fit')
ax3.axhline(y=0, color='k', linestyle='-', alpha=0.3)
ax3.axvline(x=V0_fit, color='g', linestyle='--', alpha=0.7, label=f'V₀ (P=0)')
ax3.set_xlabel('Volume (Bohr³)', fontsize=12)
ax3.set_ylabel('Pressure (kbar)', fontsize=12)
ax3.set_title('Pressure vs Volume', fontsize=14)
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Fit residuals
ax4 = axes[1, 1]
E_fit_at_data = birch_murnaghan(V_data, *popt)
residuals = (E_data - E_fit_at_data) * 1000  # mRy
ax4.bar(range(len(residuals)), residuals, color='steelblue', alpha=0.7)
ax4.axhline(y=0, color='k', linestyle='-', alpha=0.3)
ax4.set_xlabel('Data Point', fontsize=12)
ax4.set_ylabel('Residual (mRy)', fontsize=12)
ax4.set_title('Fit Residuals', fontsize=14)
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(str(WORK_DIR / 'eos_fit.png'), dpi=150, bbox_inches='tight')
plt.show()
print(f"\nFigure saved to: {WORK_DIR / 'eos_fit.png'}")

---

## 7. Update Converged Parameters

In [ ]:
# Save optimized parameters
params_file = WORKSHOP_ROOT / 'converged_parameters.json'

if params_file.exists():
    with open(params_file, 'r') as f:
        params = json.load(f)
else:
    params = {}

# Add lattice optimization results
params['celldm1_optimized_bohr'] = float(a0_fit_bohr)
params['a_optimized_angstrom'] = float(a0_fit_angstrom)
params['V0_bohr3'] = float(V0_fit)
params['E0_ry'] = float(E0_fit)
params['bulk_modulus_gpa'] = float(B0_fit_gpa)
params['bulk_modulus_derivative'] = float(B0_prime_fit)

with open(params_file, 'w') as f:
    json.dump(params, f, indent=2)

print("Updated converged parameters:")
print(json.dumps(params, indent=2))
print(f"\nSaved to: {params_file}")

---

## Summary

In this notebook, we have:

1. ✓ Understood the importance of lattice optimization
2. ✓ Performed energy vs volume calculations
3. ✓ Fitted the Birch-Murnaghan equation of state
4. ✓ Extracted equilibrium lattice parameter and bulk modulus
5. ✓ Compared with experimental values

### Key Results for Silicon (PBE)

| Property | DFT (PBE) | Experiment | Error |
|----------|-----------|------------|-------|
| Lattice parameter | {a0_fit_angstrom:.4f} Å | 5.431 Å | ~1-2% |
| Bulk modulus | {B0_fit_gpa:.1f} GPa | 97.6 GPa | varies |

### Converged Parameters Summary

| Parameter | Value | Source |
|-----------|-------|--------|
| ecutwfc | {ecutwfc} Ry | Notebook 03 |
| k-points | {kgrid}×{kgrid}×{kgrid} | Notebook 04 |
| celldm(1) | {a0_fit_bohr:.4f} Bohr | This notebook |

### Next Notebook
→ **06_Band_Structure.ipynb**: Calculate and visualize the electronic band structure